In [13]:
# Install these packages if running from colab
!pip install tensorflow-datasets --quiet
!pip install pydot --quiet
!pip install transformers --quiet

# install huggingface datasets
!pip install datasets --quiet

! pip install rouge-score nltk --quiet
! pip install huggingface_hub --quiet

In [14]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds

import sklearn as sk
import os
import nltk
from nltk.data import find

import matplotlib.pyplot as plt

import re

#let's make longer output readable without scrolling
from pprint import pprint

# the toxic parallel dataset, with rouge metric
from datasets import load_dataset, load_from_disk, load_metric, DatasetDict

In [15]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# define paths
dataset_path = 'drive/MyDrive/Colab Notebooks/w266_project_data'
csv_path = 'drive/MyDrive/Colab Notebooks/w266_project_predictions/'

In [17]:
# change these variables for different models
output_file_name = 'bart_detox_zsl.csv'
model_checkpoint = "facebook/bart-large-cnn"

In [18]:
# load the dataset
dataset = load_from_disk(dataset_path)

In [19]:
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [20]:
pd.DataFrame(dataset['train'])

,en_toxic_comment,en_neutral_comment
0,u 'd be surprised all the shit u think about w...,You would be supriesd all the things you think...
1,plenty of holocaust deniers actually believe t...,Plenty of Holocaust deniers actually believe t...
2,holy fuck i 'm so going,i'm so going
3,i 've driven by lots of cattle farms but holy ...,i 've driven by lots of cattle farms but this
4,so much better this shit to soft for me,So much better this is to soft for me.
...,...,...
17784,"but , thats exactly what a couple of you idiot...","But, that’s exactly what a couple of you have ..."
17785,he probably assessed the situation and decided...,He probably assessed the situation and decided...
17786,brah im fucked up over here .,Brah I am feeling bad over here
17787,shit with my friends and not being able to tru...,with my friends and not being able to trust th...


## using detox pretrained

In [21]:
# using detox pretrained
from transformers import BartForConditionalGeneration, AutoTokenizer
base_model_name = 'facebook/bart-base'
model_name = 'SkolkovoInstitute/bart-base-detox'
tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/558M [00:00<?, ?B/s]

In [22]:
#model.summary()

AttributeError: ignored

### Preprocess: Encode detox dataset with BART tokenizer

In [23]:
## Encode detox train_texts with BART tokenizer
max_length = 25
#max_label_length = 128

def preprocess_function(examples):
    inputs = [doc for doc in examples['en_toxic_comment']]
    model_inputs = tokenizer(inputs, max_length=max_length, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['en_neutral_comment'], max_length=max_length, truncation=True, padding=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [24]:
# test the preprocess function
preprocess_function(dataset['train'][:2])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3547: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "


{'input_ids': [[0, 257, 128, 417, 28, 3911, 70, 5, 15328, 1717, 206, 59, 77, 1717, 1236, 687, 579, 2582, 179, 89, 2], [0, 2911, 24997, 9, 18701, 43328, 3069, 4733, 888, 679, 5, 5373, 15328, 51, 224, 479, 2, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]], 'labels': [[0, 1185, 74, 28, 28836, 4458, 417, 70, 5, 383, 47, 206, 59, 77, 47, 32, 95, 2828, 89, 4, 2], [0, 16213, 24997, 9, 13903, 3069, 4733, 888, 679, 5, 383, 51, 224, 4, 2, 1, 1, 1, 1, 1, 1]]}

In [25]:
# tokenize the detox dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

### no fine-tuning, zero-shot. Try on one example

In [28]:
input_tokenized = tokenizer([dataset['test']['en_toxic_comment'][1]], return_tensors="pt").input_ids
summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)

In [29]:
prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
reference = [dataset['test']['en_neutral_comment'][1]]

In [30]:
print([dataset['test']['en_toxic_comment'][1]])
print(prediction)
print(reference)

['all she has to do is smile and keep her cool with slobbering old wrinkled hubby .']
['All she has to do is smile and keep her cool with old aged hubby']
['All she has to do is smile and keep her cool with old aged hubby']


In [31]:
rouge_results = metric.compute(predictions=prediction,
                               references=reference)
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}


## zero-shot learning (ZSL) on the test set

In [32]:
test_predictions = []
test_references = []
test_len = len(dataset['test']['en_toxic_comment'])

for i in range(test_len):
  input_tokenized = tokenizer([dataset['test']['en_toxic_comment'][i]], return_tensors="pt").input_ids
  summary_ids = model.generate(input_tokenized, num_beams=2, min_length=0, max_length=25)
  
  prediction = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  reference = [dataset['test']['en_neutral_comment'][i]]
  
  test_predictions.insert(i, (prediction))
  test_references.insert(i, (reference))

  if i % 50 == 0:
    print('complete', i, '/', test_len)
#print(len(val_references))

complete 0 / 989
complete 50 / 989
complete 100 / 989
complete 150 / 989
complete 200 / 989
complete 250 / 989
complete 300 / 989
complete 350 / 989
complete 400 / 989
complete 450 / 989
complete 500 / 989
complete 550 / 989
complete 600 / 989
complete 650 / 989
complete 700 / 989
complete 750 / 989
complete 800 / 989
complete 850 / 989
complete 900 / 989
complete 950 / 989


In [33]:
# dictionary of lists  
dict = {'test_predictions': test_predictions, 'test_references': test_references}  
       
df = pd.DataFrame(dict) 

In [34]:
# saving the output dataframe to a csv file
df.to_csv(csv_path + output_file_name, index = False) 

In [35]:
# read data from csv file
df_bart_predictions = pd.read_csv(csv_path + output_file_name)

# display data frame
print(df_bart_predictions.head())

# display shape
print(df_bart_predictions.shape)

# display columns
print(df_bart_predictions.columns)

                                    test_predictions  \
0               ['The 61 61 view must be hilarious']   
1  ['All she has to do is smile and keep her cool...   
2  ['can we apply that litmus test to muslims tha...   
3                          ['thats it , im joining']   
4           ['The residents of Davis are not good.']   

                                     test_references  
0                    ['The 6161 view must be funny']  
1  ['All she has to do is smile and keep her cool...  
2  ['can we apply that litmus test to muslims tha...  
3                        ["That's it! I'm joining."]  
4    ['The residents of Davis are not so pleasant.']  
(989, 2)
Index(['test_predictions', 'test_references'], dtype='object')


In [36]:
rouge_results = metric.compute(predictions=df_bart_predictions['test_predictions'],
                               references=df_bart_predictions['test_references'])
pprint(rouge_results, compact=True)  

{'rouge1': AggregateScore(low=Score(precision=0.8444242774238322, recall=0.8421726537016013, fmeasure=0.838288365534359), mid=Score(precision=0.8578235183048736, recall=0.8543802305165107, fmeasure=0.8509839098600782), high=Score(precision=0.8699607790236451, recall=0.8664960509517158, fmeasure=0.8632674192145228)),
 'rouge2': AggregateScore(low=Score(precision=0.74468794917538, recall=0.7395372239298071, fmeasure=0.7377073424494864), mid=Score(precision=0.7628304683678508, recall=0.757729238146549, fmeasure=0.7559570722886466), high=Score(precision=0.781327399151688, recall=0.7757977339247779, fmeasure=0.7737690537792857)),
 'rougeL': AggregateScore(low=Score(precision=0.8401812631285772, recall=0.8379490638511323, fmeasure=0.8341612623536487), mid=Score(precision=0.8539864416262537, recall=0.8505781756660429, fmeasure=0.8471150980129819), high=Score(precision=0.8664688307932231, recall=0.8631795133564052, fmeasure=0.8592907445268766)),
 'rougeLsum': AggregateScore(low=Score(precision

In [37]:
pd.DataFrame.from_dict(rouge_results)

,rouge1,rouge2,rougeL,rougeLsum
0,"(0.8444242774238322, 0.8421726537016013, 0.838...","(0.74468794917538, 0.7395372239298071, 0.73770...","(0.8401812631285772, 0.8379490638511323, 0.834...","(0.840718965836794, 0.8381885881871641, 0.8349..."
1,"(0.8578235183048736, 0.8543802305165107, 0.850...","(0.7628304683678508, 0.757729238146549, 0.7559...","(0.8539864416262537, 0.8505781756660429, 0.847...","(0.8538058787654973, 0.8503063624146698, 0.847..."
2,"(0.8699607790236451, 0.8664960509517158, 0.863...","(0.781327399151688, 0.7757977339247779, 0.7737...","(0.8664688307932231, 0.8631795133564052, 0.859...","(0.8660520459739163, 0.8629263888768194, 0.859..."


In [38]:
pd.concat({k: pd.DataFrame(v) for k, v in rouge_results.items()})

precision    recall  fmeasure
rouge1    0   0.844424  0.842173  0.838288
          1   0.857824  0.854380  0.850984
          2   0.869961  0.866496  0.863267
rouge2    0   0.744688  0.739537  0.737707
          1   0.762830  0.757729  0.755957
          2   0.781327  0.775798  0.773769
rougeL    0   0.840181  0.837949  0.834161
          1   0.853986  0.850578  0.847115
          2   0.866469  0.863180  0.859291
rougeLsum 0   0.840719  0.838189  0.834925
          1   0.853806  0.850306  0.847090
          2   0.866052  0.862926  0.859721